# Took from [here](https://www.youtube.com/watch?v=5Lxuu16_28o)


In [ ]:
from google.colab import files
files.upload()

Saving custom_dt_b32i640.pt to custom_dt_b32i640.pt


or

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%%shell
git clone https://github.com/ultralytics/yolov5
pip install -qr yolov5/requirements.txt
echo "End."

fatal: destination path 'yolov5' already exists and is not an empty directory.
End.


In [ ]:
%cd yolov5
!ls

import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile
from models.yolo import Model


%cd models
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

[Errno 2] No such file or directory: 'yolov5'
/content/yolov5/models
common.py	 hub	      yolo.py	    yolov5s.yaml
experimental.py  __init__.py  yolov5l.yaml  yolov5x.yaml
export.py	 __pycache__  yolov5m.yaml
[Errno 2] No such file or directory: 'models'
/content/yolov5/models
Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
PATH='..the .pt or .zip path'
PATH='/content/custom_dt_b32i640.pt'

#https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
#model = Model()
#model.load_state_dict(torch.load(PATH))

#See https://github.com/ultralytics/yolov5/blob/master/train.py#L88
model = Model(torch.load(PATH)['model'].yaml)
#model = torch.load(PATH)

model.eval()

Model(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv3): Conv(
       

In [ ]:
!du -h ../../*.pt

14M	../../DoorGit_DT_b32i320.pt


In [ ]:
model.modules

# 1. Fuse Layers
Sfortunatamente si possono fondere solo queste sequenze ([link](https://pytorch.org/docs/stable/torch.quantization.html#torch.quantization.fuse_modules)).

Why: [link](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjbrf_B19XvAhWMlqQKHfFFD5UQFjAAegQIAhAD&url=https%3A%2F%2Fcompas.cs.stonybrook.edu%2F~mferdman%2Fdownloads.php%2FMICRO16_Fused_Layer_CNN_Accelerators.pdf&usg=AOvVaw3X0ycxN_C9Rt6t0BVS793m)

How to fuse: [link](https://discuss.pytorch.org/t/how-to-fuse-layers-of-any-convolutional-neural-network/81173/2)

From PyTorch: [link](https://github.com/pytorch/pytorch/blob/master/torch/quantization/fuse_modules.py)

From repo (use this one): [link](https://github.com/ultralytics/yolov5/blob/4fb8cb353f7d1945e3e1b270980c883c82297d2f/models/yolo.py#L160-L169 )

In [ ]:
from torch.quantization import fuse_modules
from models.common import Conv, Bottleneck, SPP, DWConv, Focus, BottleneckCSP, Concat

flag=True

if flag:
  model.fuse() #from yolov5 repo
else:
  models_to_fuse = [['conv', 'bn']]
  models_to_fuse.append([['conv', 'bn', 'relu']])
  models_to_fuse.append([['conv', 'relu']])
  models_to_fuse.append([['linear', 'relu']])
  models_to_fuse.append([['bn', 'relu']])
  print(models_to_fuse)
  fuse_modules(model, models_to_fuse, inplace=True)

print(model)


Model(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU()
      )
      (cv3): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU()
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU()
          )
          (cv2): Conv(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU()
          )
        )
 

# 2. Quantize
Quantizzazione STATIC POST TRAINING.
(Esistono anche dynamic post-training e quantization aware training)

In [ ]:
!ls /content/images

In [ ]:
#NOT SUPPORTED!!!
from torchvision import datasets, transforms

#qnnpack --> ONLY FOR ARM, 8bit quantization
#https://github.com/pytorch/QNNPACK
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
torch.quantization.prepare(model, inplace=True)

#calibrate, not mandatory
def calibrate(model, calibration_data):
  return
calibrate(model, [])

torch.quantization.convert(model, inplace=True)

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:957: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  Returning default scale and zero point "


Model(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): QuantizedConv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): QuantizedConv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), scale=1.0, zero_point=0, padding=(1, 1))
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): QuantizedConv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): QuantizedConv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
        (act): SiLU()
      )
      (cv3): Conv(
        (conv): QuantizedConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
        (act): SiLU()
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): QuantizedConv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_p

In [ ]:
#found in export.py for compatibility in export
for k, m in model.named_modules():
  m._non_persistent_buffers_set = set()  # pytorch 1.6.0 compatibility
  if isinstance(m, models.common.Conv):  
    # assign export-friendly activations
    if isinstance(m.act, nn.Hardswish):
      m.act = Hardswish()
    elif isinstance(m.act, nn.SiLU):
      m.act = SiLU()

In [ ]:
import torch.nn as nn

import models
from utils.activations import Hardswish, SiLU

#no linear and sequential
model = torch.quantization.quantize_dynamic(model, {nn.Module, nn.SiLU, nn.Hardswish, models.common.Conv, model.modules}, dtype=torch.qint8)
#model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear, torch.nn.Sequential, torch.nn.SiLU, model.modules}, dtype=torch.qint8)

In [ ]:
model.modules

# 3. Export & optimize

In [ ]:
print(model)

In [ ]:
savename = '/content/aa.torchscript.pt'

mobile=True

#https://github.com/ultralytics/yolov5/blob/master/models/export.py#L45
if mobile:
  img = torch.zeros(1,3,320,320)

  #model.model[-1].export = False
  #y = model(img)

  ts_model = torch.jit.trace(model, img)
  ts_model_opt = optimize_for_mobile(ts_model)
  #torch.jit.save(ts_model_opt, savename)
  ts_model_opt.save(savename)
else:
  torch.save(model, savename)

!du -h $savename

# Simple export

In [ ]:
print(torch.__version__)

1.8.1+cu101


In [ ]:
%cd /content/yolov5/

!python models/export.py  --weights /content/best.pt --img 640
#!python "models/export.py"  --weights ../best_500e.pt

/content/yolov5

devi aggiungere queste righe al file export.py
riga 58 da "model.model[-1].export = not opt.grid" a "model.model[-1].export = False" 
tra riga 65 e 66 vanno inserite "from torch.utils.mobile_optimizer import optimize_for_mobile" "ts = optimize_for_mobile(ts)"

Namespace(batch_size=1, device='cpu', dynamic=False, grid=False, img_size=[640, 640], simplify=False, weights='/content/best.pt')
YOLOv5 🚀 v5.0-26-g14d2d2d torch 1.8.1+cu101 CPU

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients, 16.3 GFLOPS

TorchScript: starting export with torch 1.8.1+cu101...
./models/yolo.py:50: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:
TorchScript: export success, saved as /content/best.torchscrip